# 家計調査と小売物価統計調査

総務省統計局の消費者物価指数の[説明](https://www.stat.go.jp/data/cpi/index.html)に次のようにあります。

> 指数計算に採用している各品目のウエイトは総務省統計局実施の家計調査の結果等に基づいています。 品目の価格は総務省統計局実施の小売物価統計調査によって調査された小売価格を用いています。 結果は各種経済施策や年金の改定などに利用されています。

家計調査と小売物価統計調査を見てみましょう。

準備として、ライブラリをインポートします。

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import requests
# データ処理に用いるライブラリ
import datetime
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
#!uv pip install kaleido==0.2.1

『[Pythonではじめるオープンデータ分析](https://www.kodansha.co.jp/book/products/0000419304)』の関数をインポートします。

In [ ]:
os.getcwd(), os.path.abspath(os.path.join(os.getcwd(), "../../.."))

In [ ]:
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
    create_hierarchy_dataframe
)

API キーなどの環境変数を設定します。

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

## [家計調査](https://www.stat.go.jp/data/kakei/index.htm)

「[家計調査 家計収支編 二人以上の世帯 月次 用途分類（世帯主の年齢階級別）](https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200561&tstat=000000330001&cycle=1&tclass1=000000330001&tclass2=000000330004&statdisp_id=0002070010&tclass3val=0)」を用いて、家計収支の前年同月比を可視化します。

可視化の際は、家計調査の「[結果の引用・転載について](https://www.stat.go.jp/data/kakei/inyou.html)」の規約に従います。

e-Stat API は1 回のリクエストでの取得件数の合計が10 万件に制限されています。
まずメタ情報を取得し、データの件数を確認します。

In [ ]:
monthly_statsDataId = "0002070010"
monthly_meta = get_metainfo(appId, monthly_statsDataId)
monthly_metadata = monthly_meta["GET_META_INFO"]["METADATA_INF"]
monthly_total_num = monthly_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
monthly_total_num

メタデータから階層のデータフレームを作成します。用途分類の名称とコードの対応を確認します。

In [ ]:
household_levels = create_hierarchy_dataframe(monthly_meta, cat_key=1)
food_levels = household_levels[household_levels["用途分類階層4"]=="060_食料"]
food_levels.head()

統計データを取得します。

In [ ]:
monthly_params = {
    "lvCat01": "4",  # "用途分類"のlevelを"4"に絞る
    "cdCat02": "04",  # "世帯区分"を"二人以上の世帯のうち勤労者世帯（2000年～）"に絞る
    "cdCat03": "A00",  # "世帯主の年齢階級"を"平均"に絞る
    "cdTimeFrom": "2018000101",  # "時間軸（月次）"を2018年以降に絞る
}
monthly_data = get_statsdata(appId, monthly_statsDataId, params=monthly_params)
monthly_value = colname_to_japanese(cleansing_statsdata(monthly_data))
monthly_df = monthly_value.assign(**{
    "年": monthly_value["時間軸（月次）コード"].astype(int) // 1_000_000,
    "月": monthly_value["時間軸（月次）コード"].str[-2:].astype(int)
})[["年", "月", "用途分類", "値"]]

前年同月比の折れ線グラフをPlotly で可視化します。

In [ ]:
# 2019年から2024年まで6年分の年範囲を作成
year = 2024
year_range = range(year - 5, year + 1)
# 使用する色をPlotlyの定義済みカラーパレットから選択
yoy_colors = px.colors.qualitative.Plotly[: len(year_range)]
# X軸の目盛りテキスト用に"1月"から"12月"までの文字列を用意
months = [f"{mm}月" for mm in range(1, 13)]
# 使用する用途分類のリストを用意
yoy_cats = ["食料", "光熱・水道", "教養娯楽", "勤め先収入"]
# 2*2 のサブプロット用の行・列のペア
row_col_pair = [(i, j) for i in range(1, 3) for j in range(1, 3)]
# 2*2のサブプロットを作成
yoy_fig = make_subplots(rows=2, cols=2, subplot_titles=yoy_cats)
# 各年度ごとに各用途分類のトレースを追加
for cat, (row, col) in zip(yoy_cats, row_col_pair):
    for n, yyyy in enumerate(year_range):
        # 指定した用途分類と年に対する条件を作成
        yoy_cond = monthly_df["用途分類"] == cat
        yoy_cond &= monthly_df["年"] == yyyy
        # 最初の用途分類のトレースのみに凡例を表示
        show_legend = True if (row == 1 and col == 1) else False
        # 折れ線グラフのトレースをサブプロットに追加
        yoy_fig.add_trace(
            go.Scatter(
                x=list(range(1, 13)),
                y=monthly_df.loc[yoy_cond, "値"],
                name=f"{yyyy}年",
                marker={"color": yoy_colors[n]},
                showlegend=show_legend,  # 最初のトレースのみ凡例を表示
            ),
            row=row, col=col,
        )
        # X軸とY軸の設定を更新
        yoy_fig.update_xaxes(
            tickvals=list(range(1, 13)),  # X軸の目盛り値。1月から12月までの数値
            ticktext=months,  # X軸の目盛りテキスト。"1月"から"12月"までの文字列
            row=row, col=col,
        )
        yoy_fig.update_yaxes(
            tickformat=",.0f",
            ticksuffix="円",
            row=row, col=col,
        )
yoy_fig.update_layout(width=1000, height=800,
    legend={
        "orientation": "h",  # "h" は水平(horizontal)を意味し、凡例の項目が水平方向に並ぶ
        "x": 0.5,  # 0.5 はグラフの水平方向の中央
        "y": -0.1,  # -0.1 は、グラフ領域外の下部(グラフの下端からさらに10%下)に凡例を配置
        "xanchor": "center",  # 凡例の中央がxで指定された位置に配置
        "yanchor": "top",  # 凡例の上端がyで指定された位置に配置
    },
)
yoy_fig.show()

必要に応じて、figureを画像やHTMLで保存します。

In [ ]:
#yoy_fig.write_image("yoy_fig.png", width=1000, height=800)

In [ ]:
#yoy_fig.write_html("yoy_fig.html")

個別の前年同月比グラフを簡単に可視化するために関数を作成します。

In [ ]:
def get_kakei_statsdata(
        app_id: str, stats_id: str, cat01: str, year_from: int = 2018
    ) -> pd.DataFrame:
    params = {
        "cdCat01": cat01,
        "cdCat02": "04",  # "世帯区分"を"二人以上の世帯のうち勤労者世帯（2000年～）"に絞る
        "cdCat03": "A00",  # "世帯主の年齢階級"を"平均"に絞る
        "cdTimeFrom": f"{year_from}000000",
    }
    data = get_statsdata(app_id, stats_id, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    df = value.assign(**{
        "年": value["時間軸（月次）コード"].astype(int) // 1_000_000,
        "月": value["時間軸（月次）コード"].str[-2:].astype(int),
        "年月": lambda x: pd.to_datetime(x["年"].astype(str) + "-" + x["月"].astype(str) + "-01")
    })[['年', '月', '年月', "値"]].sort_values("年月")
    return df

In [ ]:
def create_syoy_fig(
        df: pd.DataFrame, 
        source_name: str = "",
        source_org: str = "",
        source_url: str = "",
        title=None,
        width: int = 800, 
        height: int = 400, 
        font_size: int = 18,
    ):
    # X軸の目盛りテキスト用に"1月"から"12月"までの文字列を用意
    months = [f"{mm}月" for mm in range(1, 13)]
    yoy_fig = px.line(df, x="月", y="値", color="年", markers=True)
    yoy_fig.update_layout(width=width, height=height, font={"size": font_size},
        title={"text": title, "x": 0.5, "xanchor": "center", "font": {"size": font_size}},
        xaxis={"title": None},
        yaxis={
            "tickformat": ",.0f", 
            "ticksuffix": "円", 
            "title": None,
            # "rangemode": "tozero"
        },
            margin={"b": 80}  # 下部マージンを広げて注釈のスペースを確保
        )
    yoy_fig.update_xaxes(
        tickvals=list(range(1, 13)),  # X軸の目盛り値。1月から12月までの数値
        ticktext=months,  # X軸の目盛りテキスト。"1月"から"12月"までの文字列
    )
    # 出典情報を追加
    today = datetime.date.today()
    date_str = f"{today.year}年{today.month}月{today.day}日"
    citation = f"＊「{source_name}」（{source_org}）（{source_url}）（{date_str}に利用）"
    yoy_fig.add_annotation(
        text=citation,
        xref="paper",
        yref="paper",
        x=1,  # 右端
        y=-0.15,  # グラフの下（外側）
        xanchor="right",
        yanchor="top",
        showarrow=False,
        font={"size": 10, "color": "gray"}
    )
    return yoy_fig

勤め先収入を可視化します。

In [ ]:
salary_df = get_kakei_statsdata(appId, monthly_statsDataId, "021", year_from = 2018,)
salary_fig = create_syoy_fig(
    salary_df, 
    source_name="家計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kakei/index.htm",
    title="勤め先収入"
)
salary_fig.show()

In [ ]:
#salary_fig.write_image("salary_fig.png", width=800, height=400)
#salary_fig.write_html("salary_fig.html")

食料の支出を可視化します。

In [ ]:
food_df = get_kakei_statsdata(appId, monthly_statsDataId, "060", year_from = 2018)
food_fig = create_syoy_fig(
    food_df, 
    source_name="家計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kakei/index.htm",
    title="食料(消費支出)"
)
food_fig.show()

In [ ]:
#food_fig.write_image("food_fig.png", width=800, height=400)
#food_fig.write_html("food_fig.html")

この後の消費者物価指数で調べると、米やコーヒー豆の物価が特に急上昇しています。

そこで、米の支出を可視化します。

In [ ]:
rice_df = get_kakei_statsdata(appId, monthly_statsDataId, "062", year_from = 2018)
rice_fig = create_syoy_fig(
    rice_df, 
    source_name="家計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kakei/index.htm",
    title="米(消費支出)"
)
rice_fig.show()

In [ ]:
#rice_fig.write_image("rice_fig.png", width=800, height=400)
#rice_fig.write_html("rice_fig.html")

コーヒーの支出を可視化します。

In [ ]:
coffee_df = get_kakei_statsdata(appId, monthly_statsDataId, "095", year_from = 2018)
coffee_fig = create_syoy_fig(
    coffee_df, 
    source_name="家計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kakei/index.htm",
    title="コーヒー(消費支出)"
)
coffee_fig.show()

In [ ]:
#coffee_fig.write_image("coffee_fig.png", width=800, height=400)
#coffee_fig.write_html("coffee_fig.html")

## [小売物価統計調査](https://www.stat.go.jp/data/kouri/doukou/2.html)

小売物価統計調査は家計調査同様に国の基幹統計です。
[基幹統計](https://www.soumu.go.jp/toukei_toukatsu/index/seido/1-3k.htm)は、公的統計のうち、日本国(総務大臣)が重要と指定する統計です。
公的統計の種類は非常に多いため、書籍『[Pythonではじめるオープンデータ分析―経済統計の取得からデータハンドリング、可視化、分析まで―](https://www.kodansha.co.jp/book/products/0000419304)』では基幹統計を中心に紹介しましたが、基幹統計だけでも54種類あるため、小売物価統計は書籍では取り上げることができませんでした。

「小売物価統計 [主要品目の都市別小売価格－都道府県庁所在市及び人口15万以上の市(2000年1月～)](https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200571&tstat=000000680001&cycle=1&statdisp_id=0003421913&tclass1val=0)」を用いて、物価の前年同月比を可視化します。

可視化の際は、小売物価統計調査の「[結果の引用・利用上の注意](https://www.stat.go.jp/data/kouri/doukou/2.html)」の規約に従います。

e-Stat API は1 回のリクエストでの取得件数の合計が10 万件に制限されています。
まずメタ情報を取得し、データの件数を確認します。

In [ ]:
kouri_statsDataId = "0003421913"
kouri_meta = get_metainfo(appId, kouri_statsDataId)
kouri_metadata = kouri_meta["GET_META_INFO"]["METADATA_INF"]
kouri_total_num = kouri_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
kouri_total_num

各カテゴリのメタデータを確認します。

In [ ]:
kouri_metadata['CLASS_INF']['CLASS_OBJ'][0]

In [ ]:
kouri_metadata['CLASS_INF']['CLASS_OBJ'][1]

In [ ]:
pd.DataFrame(kouri_metadata['CLASS_INF']['CLASS_OBJ'][2]['CLASS'])

In [ ]:
pd.DataFrame(kouri_metadata['CLASS_INF']['CLASS_OBJ'][2]['CLASS'])["@name"].values[:10]

In [ ]:
pd.DataFrame(kouri_metadata['CLASS_INF']['CLASS_OBJ'][3]['CLASS']).head(30)

In [ ]:
pd.DataFrame(kouri_metadata['CLASS_INF']['CLASS_OBJ'][3]['CLASS'])["@name"].values

In [ ]:
pd.DataFrame(kouri_metadata['CLASS_INF']['CLASS_OBJ'][4]['CLASS'])

品目を指定して、時系列データを取得する関数を作成します。

In [ ]:
def get_kouri_statsdata(
        app_id: str, stats_id: str, cat02: str, year_from: int = 2018
    ) -> pd.DataFrame:
    params = {
        "cdCat02": cat02,
        "cdArea": "13100",  # 特別区部
        "cdTimeFrom": f"{year_from}000000",
    }
    data = get_statsdata(app_id, stats_id, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    df = value.assign(**{
        "年": value["時間軸（月）コード"].astype(int) // 1_000_000,
        "月": value["時間軸（月）コード"].str[-2:].astype(int),
        "年月": lambda x: pd.to_datetime(x["年"].astype(str) + "-" + x["月"].astype(str) + "-01")
    })[['年', '月', '年月', "値"]].sort_values("年月")
    return df

この後の消費者物価指数で調べると、米やコーヒー豆の物価が特に急上昇しています。

米の物価を可視化します。

In [ ]:
koshihikari_df = get_kouri_statsdata(appId, kouri_statsDataId, "01001", year_from = 2018)
koshihikari_fig = create_syoy_fig(
    koshihikari_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="うるち米(単一原料米,「コシヒカリ」)(小売物価)"
)
koshihikari_fig.show()

In [ ]:
#koshihikari_fig.write_image("koshihikari_fig.png", width=800, height=400)
#koshihikari_fig.write_html("koshihikari_fig.html")

コーヒーについては、コーヒー豆、インスタントコーヒー、コーヒー（外食）を取得します。

In [ ]:
coffee_beans_df = get_kouri_statsdata(appId, kouri_statsDataId, "01922", year_from = 2018)
coffee_beans_fig = create_syoy_fig(
    coffee_beans_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="コーヒー豆(小売物価)"
)
coffee_beans_fig.show()

In [ ]:
instant_coffee_df = get_kouri_statsdata(appId, kouri_statsDataId, "01921", year_from = 2018)
instant_coffee_fig = create_syoy_fig(
    instant_coffee_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="インスタントコーヒー(小売物価)"
)
instant_coffee_fig.show()

In [ ]:
eatout_coffee_df = get_kouri_statsdata(appId, kouri_statsDataId, "02162", year_from = 2018)
eatout_coffee_fig = create_syoy_fig(
    eatout_coffee_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="コーヒー(外食)(喫茶店)(小売物価)"
)
eatout_coffee_fig.show()

In [ ]:
#coffee_beans_fig.write_image("coffee_beans_fig.png", width=800, height=400)
#coffee_beans_fig.write_html("coffee_beans_fig.html")
#instant_coffee_fig.write_image("instant_coffee_fig.png", width=800, height=400)
#instant_coffee_fig.write_html("instant_coffee_fig.html")
#eatout_coffee_fig.write_image("eatout_coffee_fig.png", width=800, height=400)
#eatout_coffee_fig.write_html("eatout_coffee_fig.html")

アイスクリームはよく夏の時期にe-Statで検索されるようです。

In [ ]:
ice_df = get_kouri_statsdata(appId, kouri_statsDataId, "01782", year_from = 2018)
ice_fig = create_syoy_fig(
    ice_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="アイスクリーム(小売物価)"
)
ice_fig.show()

In [ ]:
#ice_fig.write_image("ice_fig.png", width=800, height=400)
#ice_fig.write_html("ice_fig.html")

ビッグマック指数とは、[Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%93%E3%83%83%E3%82%B0%E3%83%9E%E3%83%83%E3%82%AF%E6%8C%87%E6%95%B0)によると、マクドナルドで販売されているビッグマック1個の価格を比較することで得られる各国の経済力を測るための指数です。
ここでは、ビッグマックではなく、ハンバーガー（外食）の物価を可視化します。

In [ ]:
hamburger_df = get_kouri_statsdata(appId, kouri_statsDataId, "02135", year_from = 2018)
hamburger_fig = create_syoy_fig(
    hamburger_df, 
    source_name="小売物価統計調査結果",
    source_org="総務省統計局",
    source_url="https://www.stat.go.jp/data/kouri/index.htm",
    title="ハンバーガー(外食)(小売物価)"
)
hamburger_fig.show()

In [ ]:
#hamburger_fig.write_image("hamburger_fig.png", width=800, height=400)
#hamburger_fig.write_html("hamburger_fig.html")

## 消費者物価指数

[消費者物価指数（2020年基準）](https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200573&tstat=000001150147&cycle=0&statdisp_id=0003427113&tclass1val=0)は、書籍『[Pythonではじめるオープンデータ分析―経済統計の取得からデータハンドリング、可視化、分析まで―](https://www.kodansha.co.jp/book/products/0000419304)』の第10章「国際統計・長期経済統計の取得・可視化・分析」で説明しています。
書籍では、総合指数を取り上げましたが、ここでは"食料"などのカテゴリの物価指数を扱います。

可視化の際は、総務省統計局の「[サイトの利用について](https://www.stat.go.jp/info/riyou.html)」の規約に従います。

e-Stat API は1 回のリクエストでの取得件数の合計が10 万件に制限されています。
まずメタ情報を取得し、データの件数を確認します。

In [ ]:
cpi_statsDataId = "0003427113"
cpi_meta = get_metainfo(appId, cpi_statsDataId)
cpi_metadata = cpi_meta["GET_META_INFO"]["METADATA_INF"]
cpi_total_num = cpi_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
cpi_total_num

カテゴリごとにメタデータを確認します。

In [ ]:
cpi_metadata['CLASS_INF']['CLASS_OBJ'][0]['CLASS']

In [ ]:
cpi_cat_df = pd.DataFrame(cpi_metadata['CLASS_INF']['CLASS_OBJ'][1]['CLASS'])
cpi_cat_df

In [ ]:
cpi_cat_df[cpi_cat_df["@level"]=="1"].head()

In [ ]:
cpi_cat_df.loc[cpi_cat_df["@level"]=="6", "@name"].values[:5]

In [ ]:
pd.DataFrame(cpi_metadata['CLASS_INF']['CLASS_OBJ'][2]['CLASS'])

In [ ]:
pd.DataFrame(cpi_metadata['CLASS_INF']['CLASS_OBJ'][3]['CLASS'])

特定の年月のすべての品目の物価を取得する関数を作成します。

In [ ]:
def get_cpi_statsdata_by_lv(
        app_id: str, stats_id: str, lvcat01: str, yyyy: str = "2024", mm: str = "00"
    ) -> pd.DataFrame:
    params = {
        "cdTab": "1",
        "lvCat01": lvcat01,
        "cdArea": "00000",  # 全国
        "cdTime": f"{yyyy}00{mm}{mm}",
    }
    data = get_statsdata(app_id, stats_id, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    df = value.assign(**{
        "年": value["時間軸（年・月）コード"].astype(int) // 1_000_000,
    })[['年', '2020年基準品目', "値"]].sort_values("年")
    return df

品目ごとに直近2025年7月とコロナ禍前の2019年7月の物価を比較します。

In [ ]:
cpi_lv1_df_25 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "1", yyyy = "2025", mm = "07")
cpi_lv1_df_19 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "1", yyyy = "2019", mm = "07")

cpi_lv6_df_25 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "6", yyyy = "2025", mm = "07")
cpi_lv6_df_19 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "6", yyyy = "2019", mm = "07")

In [ ]:
cpi_lv1_df = pd.concat([cpi_lv1_df_25, cpi_lv1_df_19]).set_index(["2020年基準品目", "年"]).unstack()
cpi_lv1_sort = (cpi_lv1_df
    .set_axis(cpi_lv1_df.columns.levels[1], axis=1)
    .assign(**{"物価上昇率": lambda x: x[2025] / x[2019]})
    .sort_values("物価上昇率", ascending=False)
)
cpi_lv1_sort.head(30)

In [ ]:
cpi_lv6_df = pd.concat([cpi_lv6_df_25, cpi_lv6_df_19]).set_index(["2020年基準品目", "年"]).unstack()
cpi_lv6_sort = (cpi_lv6_df
    .set_axis(cpi_lv6_df.columns.levels[1], axis=1)
    .assign(**{"物価上昇率": lambda x: x[2025] / x[2019]})
    .sort_values("物価上昇率", ascending=False)
)
cpi_lv6_sort.head(30)

In [ ]:
#!uv pip install tabulate

In [ ]:
print(cpi_lv6_sort.head(20).to_markdown())

より細分化した品目についても比較します。

In [ ]:
cpi_lv6_df_2025 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "6", yyyy = "2025", mm = "07")
cpi_lv6_df_2019 = get_cpi_statsdata_by_lv(appId, cpi_statsDataId, "6", yyyy = "2019", mm = "07")
cpi_lv6_df = pd.concat([cpi_lv6_df_2025, cpi_lv6_df_2019]).set_index(["2020年基準品目", "年"]).unstack()
cpi_lv6_sort = (cpi_lv6_df
    .set_axis(cpi_lv6_df.columns.levels[1], axis=1)
    .assign(**{"物価上昇率": lambda x: x[2025] / x[2019]})
    .sort_values("物価上昇率", ascending=False)
)
cpi_lv6_sort.head(30)

続いて、品目を指定して、時系列データを取得する関数を作成します。

In [ ]:
def get_cpi_statsdata(
        app_id: str, stats_id: str, cat01: str, year_from: int = 2018
    ) -> pd.DataFrame:
    params = {
        "cdTab": "1",
        "cdCat01": cat01,
        "cdArea": "00000",  # 全国
        "cdTimeFrom": f"{year_from}000000",
    }
    data = get_statsdata(app_id, stats_id, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    value["時間軸（年・月）コード"].str[-2:]!="00"
    df = value[value["時間軸（年・月）コード"].str[-2:]!="00"].assign(**{
        "年": value["時間軸（年・月）コード"].astype(int) // 1_000_000,
        "月": value["時間軸（年・月）コード"].str[-2:].astype(int),
        "年月": lambda x: pd.to_datetime(x["年"].astype(str) + "-" + x["月"].astype(str) + "-01")
    })[['年', '月', '年月', '2020年基準品目', "値"]].sort_values("年月")
    return df

米やコーヒーの物価上昇率が高くなっています。
そこで、米（うるち米A）のデータを取得します。
コーヒーについては、コーヒー豆、インスタントコーヒー、コーヒー（外食）を取得します。
比較のために食料と総合指数も取得します。

In [ ]:
cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "0001", year_from = 2018)
food_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "0002", year_from = 2018)
rice_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "1001", year_from = 2018)
coffee_beans_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "1922", year_from = 2018)
instant_coffee_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "1921", year_from = 2018)
eatout_coffee_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "2162", year_from = 2018)
hamburger_cpi_df = get_cpi_statsdata(appId, cpi_statsDataId, "2135", year_from = 2018)  # おまけ

cpi_concat = pd.concat([
    cpi_df, 
    food_cpi_df, 
    rice_cpi_df, 
    coffee_beans_cpi_df, 
    instant_coffee_cpi_df, 
    eatout_coffee_cpi_df, 
    hamburger_cpi_df
])
cpi_concat.head()

2018年から2025年までの消費者物価指数の推移を可視化します。

In [ ]:
source_name = "「消費者物価指数」"
source_org = "総務省統計局"
source_url = "https://www.stat.go.jp/data/cpi/index.htm"
cpi_fig = px.line(cpi_concat, x="年月", y="値", color="2020年基準品目")
cpi_fig.update_layout(width=1000, height=400, font={"size": 18},
    title={"text": "消費者物価指数", "x": 0.3, "xanchor": "center", "font": {"size": 18}},
    xaxis={"title": None, "ticksuffix": "年"},
    yaxis={
        "tickformat": ",.0f", 
        "title": None,
        # "rangemode": "tozero"
    },
    margin={"b": 80}  # 下部マージンを広げて注釈のスペースを確保
)
# 出典情報を追加
today = datetime.date.today()
date_str = f"{today.year}年{today.month}月{today.day}日"
citation = f"＊「{source_name}」（{source_org}）（{source_url}）（{date_str}に利用）"
cpi_fig.add_annotation(
    text=citation,
    xref="paper",
    yref="paper",
    x=1,  # 右端
    y=-0.15,  # グラフの下（外側）
    xanchor="right",
    yanchor="top",
    showarrow=False,
    font={"size": 10, "color": "gray"}
)
cpi_fig.show()

In [ ]:
#cpi_fig.write_image("cpi_fig.png", width=1000, height=400)
#cpi_fig.write_html("cpi_fig.html")

## 家計調査と小売物価統計調査の組み合わせ


In [ ]:
# グラフの作成
rice_fig2 = go.Figure()
# 家計調査のデータ（左軸）
rice_fig2.add_trace(go.Scatter(
    x=rice_df['年月'],
    y=rice_df['値'],
    name='家計調査: 米',
    yaxis='y'
))
# 小売物価統計調査のデータ（右軸）
rice_fig2.add_trace(go.Scatter(
    x=koshihikari_df['年月'],
    y=koshihikari_df['値'],
    name='小売物価統計調査: うるち米',
    yaxis='y2'
))
# レイアウトの設定
rice_fig2.update_layout(width=1000, height=400, font={"size": 18},
    title={"text": "家計調査と小売物価統計調査の米の推移（2軸グラフ）", "x": 0.3, "xanchor": "center", "font": {"size": 18}},
    xaxis={"title": None, "ticksuffix": "年"},
    yaxis={"title": "家計調査（円）", "tickformat": ",.0f"},
    yaxis2={
        "title": "小売物価統計（円）",
        "tickformat": ",.0f", 
        "overlaying": "y",
        "side": "right"
    },
    legend={"x": 0.01, "y": 0.99},
    margin={"b": 80},  # 下部マージンを広げて注釈のスペースを確保
    hovermode="x unified",
)
# 出典情報を追加
today = datetime.date.today()
date_str = f"{today.year}年{today.month}月{today.day}日"
citation = f"＊「家計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kakei/index.htm）および<br>「小売物価統計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kouri/index.htm）（{date_str}に利用）"
rice_fig2.add_annotation(
    text=citation,
    xref="paper",
    yref="paper",
    x=1.0,  # 右端
    y=-0.15,  # グラフの下（外側）
    xanchor="right",
    yanchor="top",
    showarrow=False,
    font={"size": 10, "color": "gray"},
    align="left",
)
rice_fig2.show()

In [ ]:
#rice_fig2.write_image("rice_fig2.png", width=1000, height=400)
#rice_fig2.write_html("rice_fig2.html")

In [ ]:
# グラフの作成
coffee_beans_fig2 = go.Figure()
# 家計調査のデータ（左軸）
coffee_beans_fig2.add_trace(go.Scatter(
    x=coffee_df['年月'],
    y=coffee_df['値'],
    name='家計調査: コーヒー',
    yaxis='y'
))
# 小売物価統計調査のデータ（右軸）
coffee_beans_fig2.add_trace(go.Scatter(
    x=coffee_beans_df['年月'],
    y=coffee_beans_df['値'],
    name='小売物価統計調査: コーヒー豆',
    yaxis='y2'
))
# レイアウトの設定
coffee_beans_fig2.update_layout(width=1000, height=400, font={"size": 18},
    title={"text": "家計調査と小売物価統計調査のコーヒーの推移（2軸グラフ）", "x": 0.3, "xanchor": "center", "font": {"size": 18}},
    xaxis={"title": None, "ticksuffix": "年"},
    yaxis={"title": "家計調査（円）", "tickformat": ",.0f"},
    yaxis2={
        "title": "小売物価統計（円）",
        "tickformat": ",.0f", 
        "overlaying": "y",
        "side": "right"
    },
    legend={"x": 0.01, "y": 0.99},
    margin={"b": 80},  # 下部マージンを広げて注釈のスペースを確保
    hovermode="x unified",
)
# 出典情報を追加
today = datetime.date.today()
date_str = f"{today.year}年{today.month}月{today.day}日"
citation = f"＊「家計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kakei/index.htm）および<br>「小売物価統計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kouri/index.htm）（{date_str}に利用）"
coffee_beans_fig2.add_annotation(
    text=citation,
    xref="paper",
    yref="paper",
    x=1.0,  # 右端
    y=-0.15,  # グラフの下（外側）
    xanchor="right",
    yanchor="top",
    showarrow=False,
    font={"size": 10, "color": "gray"},
    align="left",
)
coffee_beans_fig2.show()

In [ ]:
#coffee_beans_fig2.write_image("coffee_beans_fig2.png", width=1000, height=400)
#coffee_beans_fig2.write_html("coffee_beans_fig2.html")

In [ ]:
# グラフの作成
eatout_coffee_fig2 = go.Figure()
# 家計調査のデータ（左軸）
eatout_coffee_fig2.add_trace(go.Scatter(
    x=coffee_df['年月'],
    y=coffee_df['値'],
    name='家計調査: コーヒー',
    yaxis='y'
))
# 小売物価統計調査のデータ（右軸）
eatout_coffee_fig2.add_trace(go.Scatter(
    x=eatout_coffee_df['年月'],
    y=eatout_coffee_df['値'],
    name='小売物価統計調査: コーヒー（外食）',
    yaxis='y2'
))
# レイアウトの設定
eatout_coffee_fig2.update_layout(width=1000, height=400, font={"size": 18},
    title={"text": "家計調査と小売物価統計調査のコーヒーの推移（2軸グラフ）", "x": 0.3, "xanchor": "center", "font": {"size": 18}},
    xaxis={"title": None, "ticksuffix": "年"},
    yaxis={"title": "家計調査（円）", "tickformat": ",.0f"},
    yaxis2={
        "title": "小売物価統計（円）",
        "tickformat": ",.0f", 
        "overlaying": "y",
        "side": "right"
    },
    legend={"x": 0.01, "y": 0.99},
    margin={"b": 80},  # 下部マージンを広げて注釈のスペースを確保
    hovermode="x unified",
)
# 出典情報を追加
today = datetime.date.today()
date_str = f"{today.year}年{today.month}月{today.day}日"
citation = f"＊「家計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kakei/index.htm）および<br>「小売物価統計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kouri/index.htm）（{date_str}に利用）"
eatout_coffee_fig2.add_annotation(
    text=citation,
    xref="paper",
    yref="paper",
    x=1.0,  # 右端
    y=-0.15,  # グラフの下（外側）
    xanchor="right",
    yanchor="top",
    showarrow=False,
    font={"size": 10, "color": "gray"},
    align="left",
)
eatout_coffee_fig2.show()

In [ ]:
#eatout_coffee_fig2.write_image("eatout_coffee_fig2.png", width=1000, height=400)
#eatout_coffee_fig2.write_html("eatout_coffee_fig2.html")

In [ ]:
# グラフの作成
coffee_fig2 = go.Figure()
# 家計調査のデータ（左軸）
coffee_fig2.add_trace(go.Scatter(
    x=coffee_df['年月'],
    y=coffee_df['値'],
    name='家計調査: コーヒー',
    yaxis='y'
))
# 小売物価統計調査のデータ（右軸）
coffee_fig2.add_trace(go.Scatter(
    x=coffee_beans_df['年月'],
    y=coffee_beans_df['値'],
    name='小売物価統計調査: コーヒー豆',
    yaxis='y2'
))
coffee_fig2.add_trace(go.Scatter(
    x=eatout_coffee_df['年月'],
    y=eatout_coffee_df['値'],
    name='小売物価統計調査: コーヒー（外食）',
    yaxis='y2'
))
# レイアウトの設定
coffee_fig2.update_layout(width=1000, height=500, font={"size": 18},
    title={"text": "家計調査と小売物価統計調査のコーヒーの推移（2軸グラフ）", "x": 0.3, "xanchor": "center", "font": {"size": 18}},
    xaxis={"title": None, "ticksuffix": "年"},
    yaxis={"title": "家計調査（円）", "tickformat": ",.0f"},
    yaxis2={
        "title": "小売物価統計（円）",
        "tickformat": ",.0f", 
        "overlaying": "y",
        "side": "right"
    },
    legend={"x": 0.05, "y": 1.1},
    margin={"b": 80},  # 下部マージンを広げて注釈のスペースを確保
    hovermode="x unified",
)
# 出典情報を追加
today = datetime.date.today()
date_str = f"{today.year}年{today.month}月{today.day}日"
citation = f"＊「家計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kakei/index.htm）および<br>「小売物価統計調査結果」（総務省統計局）（https://www.stat.go.jp/data/kouri/index.htm）（{date_str}に利用）"
coffee_fig2.add_annotation(
    text=citation,
    xref="paper",
    yref="paper",
    x=1.0,  # 右端
    y=-0.15,  # グラフの下（外側）
    xanchor="right",
    yanchor="top",
    showarrow=False,
    font={"size": 10, "color": "gray"},
    align="left",
)
coffee_fig2.show()